In [ ]:
import asyncio
import nest_asyncio
nest_asyncio.apply()

from poke_env.player_configuration import PlayerConfiguration
from poke_env.player.random_player import RandomPlayer
from poke_env.player.baselines import SimpleHeuristicsPlayer
from poke_env.server_configuration import ShowdownServerConfiguration
from poke_env.player.utils import cross_evaluate

from tabulate import tabulate

%load_ext autoreload
%autoreload 2

In [ ]:
#Challenge A simple Heuristic Player on Showdown for fun
player = SimpleHeuristicsPlayer(
    player_configuration=PlayerConfiguration("JoeNextLine", "underground"),
    server_configuration=ShowdownServerConfiguration,
)

await player.send_challenges("meatout", n_challenges=1)


In [ ]:
import json
from dotmap import DotMap 


from pokebot.bots.state_engine import SimpleStateEngine, HeuristicStateEngine
from pokebot.bots.bot import BotPlayer
from pokebot.models.dqn import Model
from pokebot.trainers.trainer import SimpleDQNTrainer

In [ ]:
# Load hparams from file, this is where you would specify DQNAgent and Policy hyperparamters.
hparams = DotMap(json.load(open('./hparams.json', 'r')))
p_dict = hparams.policy
a_dict = hparams.agent

print((p_dict))

In [ ]:
player = BotPlayer(
    player_configuration=PlayerConfiguration("test2",None),
#     log_level=10,
    state_engine=HeuristicStateEngine()   #Change StateEngine here, (Demo SimpleStateEngine(10))
    )

model = Model(player)

In [ ]:
trainer = SimpleDQNTrainer(player, model, p_dict, a_dict)

In [ ]:
# Pick an agent to train against
# await trainer.train(opponent=RandomPlayer(battle_format="gen8randombattle"))

await trainer.train(opponent=SimpleHeuristicsPlayer(battle_format="gen8randombattle"))

In [ ]:
from poke_env.player.baselines import MaxBasePowerPlayer, SimpleHeuristicsPlayer

opponents = [RandomPlayer(battle_format="gen8randombattle"),
            MaxBasePowerPlayer(battle_format="gen8randombattle"),
             SimpleHeuristicsPlayer(battle_format="gen8randombattle")]

await trainer.evaluate(opponents)

In [ ]:
# Test agent against yourself

trainer.player = BotPlayer(
    player_configuration=PlayerConfiguration("JoeNextLine","underground"),
    server_configuration=ShowdownServerConfiguration,
#     log_level=10,
    state_engine=SimpleStateEngine(10)
    )

asyncio.get_event_loop().run_until_complete(trainer.battle_human("meatout"))

In [ ]:
#Save current player

trainer.agent.save_weights("../data/agents/bad.hdf5")

In [ ]:
#Load agent

trainer.agent.load_weights("../data/agents/bad.hdf5")